In [1]:
!pip install -U sentence-transformers
!pip install chromadb
!pip install openai

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

c:\Users\JORGE-PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import subprocess

# Ejecutar el comando desde Python
subprocess.call(["gsutil", "-m", "cp", "-r", "gs://zaia-health-dev_pocs/etl/typeSideEffect", "C:\Developer_1.0\Proyecto_GCP\\"], shell=True)

<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\JORGE-PC\AppData\Local\Temp\ipykernel_2780\17150008.py:4: SyntaxWarning: invalid escape sequence '\D'
  subprocess.call(["gsutil", "-m", "cp", "-r", "gs://zaia-health-dev_pocs/etl/typeSideEffect", "C:\Developer_1.0\Proyecto_GCP\\"], shell=True)


0

In [10]:
df = pd.read_csv('C:\\Developer_1.0\\Proyecto_GCP\\typeSideEffect\\2024_03_07_1709771913793_0.csv')

In [11]:
df.head()

,_airbyte_ab_id,_airbyte_emitted_at,_ab_cdc_deleted_at,_ab_cdc_lsn,_ab_cdc_updated_at,comment,groupSideEffect,value
0,9549724d-2608-483d-84ab-d5315ecb7008,1709771914974,NaN,NaN,NaN,"{""en"": ""Thinning hair"", ""es"": ""Adelgazamiento ...",general,thinninghair
1,52fb3369-cf94-49c9-bfb9-362c51f3785b,1709771914974,NaN,NaN,NaN,"{""en"": ""Low red blood cell count"", ""es"": ""Recu...",general,lowwhitebloodcellcount
2,d17852a0-29b8-4edb-96ce-5ed3645556a8,1709771914974,NaN,NaN,NaN,"{""en"": ""Muscle pain"", ""es"": ""Dolor muscular""}",general,musclepain
3,9454e58d-d5d7-4f70-8951-98b192d8cca8,1709771914974,NaN,NaN,NaN,"{""en"": ""Confusion"", ""es"": ""Confusión""}",general,confusion
4,2639a983-4b45-4a90-ad34-72c8208262e8,1709771914974,NaN,NaN,NaN,"{""en"": ""Painful swelling in legs"", ""es"": ""Hinc...",general,painfullegs


In [12]:
# Divide la columna "comment" por coma y expandela en dos nuevas columnas
nuevos_comentarios = df['comment'].str.split(',', expand=True)

# Añade las nuevas columnas al DataFrame (opcional)
df = pd.concat([df, nuevos_comentarios], axis=1)

#elimina colummnas que se crearon de mas
df = df.drop([2,3,4,5,6,7,8,9,10],axis=1)

#renombra las 2 columnas nuevas
df = df.rename(columns={0: 'ingles', 1: 'español'})

#modifica la columna ingles y la normaliza
df['ingles'] = df['ingles'].str.strip('{')
df['ingles'] = df['ingles'].str.replace('["]', '', regex=True)
df['ingles'] = df['ingles'].str[4:]

#modifica la columna español y la normaliza
df['español'] = df['español'].str.strip('}')
df['español'] = df['español'].str.replace('["]', '', regex=True)
df['español'] = df['español'].str[5:]

#elimina la columna vieja
df = df.drop('comment', axis=1)

# muestra imprime
df.head()

,_airbyte_ab_id,_airbyte_emitted_at,_ab_cdc_deleted_at,_ab_cdc_lsn,_ab_cdc_updated_at,groupSideEffect,value,ingles,español
0,9549724d-2608-483d-84ab-d5315ecb7008,1709771914974,NaN,NaN,NaN,general,thinninghair,Thinning hair,Adelgazamiento del cabello
1,52fb3369-cf94-49c9-bfb9-362c51f3785b,1709771914974,NaN,NaN,NaN,general,lowwhitebloodcellcount,Low red blood cell count,Recuento bajo de glóbulos blancos
2,d17852a0-29b8-4edb-96ce-5ed3645556a8,1709771914974,NaN,NaN,NaN,general,musclepain,Muscle pain,Dolor muscular
3,9454e58d-d5d7-4f70-8951-98b192d8cca8,1709771914974,NaN,NaN,NaN,general,confusion,Confusion,Confusión
4,2639a983-4b45-4a90-ad34-72c8208262e8,1709771914974,NaN,NaN,NaN,general,painfullegs,Painful swelling in legs,Hinchazón dolorosa en piernas


In [9]:
df['text'] = df.apply(lambda x : x['ingles']+' '+x['español'], axis=1)

In [13]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [15]:
embeddings = model.encode(df['text'],batch_size=64,show_progress_bar=True)

Batches: 100%|██████████| 203/203 [00:28<00:00,  7.08it/s]


In [16]:
df['embeddings'] = embeddings.tolist()

df['ids'] = df.index
df['ids'] = df['ids'].astype('str')

df

In [19]:
import chromadb
from chromadb.utils import embedding_functions

In [24]:
#no se usa el modelo de openia
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key='xxxxxxxx',
    model_name = 'text-embedding-ada-002'
)

#se usa este de sentence y asi se declara el modelo
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = 'all-MiniLM-L6-v2')

In [25]:
#cargamos la base de dato en la ruta que escojemos en local
chroma_client = chromadb.Client()
client_persistent = chromadb.PersistentClient(path= '/content/data_embeddings/')

In [26]:
#creamos una coleccion dentro de la base de datos en la variable db
db = client_persistent.create_collection(name = 'prueba23',embedding_function=sentence_transformer_ef)

In [ ]:
#db. add(
#    ids=df['ids'].tolist(),
#    embeddings=df['embeddings'].tolist(),
#    metadatas=df.drop(['ids','embeddings', 'text'],axis=1).to_dict('records')
#)

In [33]:
#se cargan los datos embeddings de df a la base de datos croma

batch_size = 5000  # Tamaño de lote deseado

# Obtener el número total de elementos
total_elements = len(df)

# Iterar sobre los datos en lotes más pequeños
for i in range(0, total_elements, batch_size):
    batch_ids = df['ids'].tolist()[i:i+batch_size]
    batch_embeddings = df['embeddings'].tolist()[i:i+batch_size]
    batch_metadatas = df.drop(['ids','embeddings', 'español'], axis=1).iloc[i:i+batch_size].to_dict('records')
    
    # Agregar el lote a la base de datos
    db.add(ids=batch_ids, embeddings=batch_embeddings, metadatas=batch_metadatas)


Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
In

In [34]:
db.peek(1)

{'ids': ['0'],
 'embeddings': [[-0.06877586990594864,
   0.03689126297831535,
   -0.043864089995622635,
   0.06560630351305008,
   -0.14996929466724396,
   -0.0039448305033147335,
   0.10782977938652039,
   0.0200058501213789,
   0.019834313541650772,
   -0.023636862635612488,
   0.08541340380907059,
   -0.0776071771979332,
   -0.04071091488003731,
   -0.02044869400560856,
   -0.005111237056553364,
   0.015217185951769352,
   0.020789561793208122,
   0.053491607308387756,
   0.002738052746281028,
   0.011644699610769749,
   0.0448576845228672,
   -0.03019784577190876,
   -0.053446508944034576,
   0.08977582305669785,
   -0.027542471885681152,
   0.00508577935397625,
   -0.033683422952890396,
   0.07151708751916885,
   -0.05357848480343819,
   -0.09195700287818909,
   -0.03149014711380005,
   0.05698387324810028,
   0.11995499581098557,
   -0.076809361577034,
   0.02961866557598114,
   0.032636772841215134,
   0.04581737145781517,
   -0.006675372365862131,
   0.08764353394508362,
   0.0

In [30]:
db_no_embeddings = client_persistent.create_collection(name='no_prueba33',embedding_function=sentence_transformer_ef)

In [31]:
#se cargan los datos no embeddings de df a la base de datos croma se vuelven # embeddings automaticamente
batch_size = 5000  # Tamaño de lote deseado

# Obtener el número total de elementos
total_elements = len(df)

# Iterar sobre los datos en lotes más pequeños
for i in range(0, total_elements, batch_size):
    batch_ids = df['ids'].tolist()[i:i+batch_size]
    batch_documents = df['text'].tolist()[i:i+batch_size]
    batch_metadatas = df.drop(['ids','embeddings', 'español'], axis=1).iloc[i:i+batch_size].to_dict('records')
    
    # Agregar el lote a la base de datos
    db_no_embeddings.add(ids=batch_ids, documents=batch_documents, metadatas=batch_metadatas)


KeyError: 'text'

In [ ]:
#db_no_embeddings.add(
#     ids=df['ids'].tolist(),
#    documents=df['text'].tolist(),
#    metadatas=df.drop(['ids','embeddings', 'text'],axis=1).to_dict('records')
#)

In [32]:
db_no_embeddings.peek(1)


{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [24]:
db_no_embeddings.delete(ids=['0'])

In [25]:
db_no_embeddings.peek(1)

{'ids': ['1'],
 'embeddings': [[0.012650537304580212,
   0.028314044699072838,
   -0.09205877035856247,
   0.049229782074689865,
   -0.1484849601984024,
   0.04328172653913498,
   0.012780523858964443,
   0.13338840007781982,
   0.014440570957958698,
   -0.06551530212163925,
   0.019370220601558685,
   -0.09241525083780289,
   -0.032548751682043076,
   -0.06152665615081787,
   -0.04208356514573097,
   -0.03350357338786125,
   0.08670338243246078,
   0.02737438678741455,
   -0.06906457245349884,
   -0.03324289619922638,
   0.0211638230830431,
   -0.03538723289966583,
   0.015281730331480503,
   -0.008857027627527714,
   -0.006638719700276852,
   -0.06249503046274185,
   -0.016262512654066086,
   -0.04419407248497009,
   -0.04717760160565376,
   -0.042209442704916,
   0.010351029224693775,
   0.024540865793824196,
   0.14279240369796753,
   -0.009285812266170979,
   0.04980625584721565,
   0.005531355272978544,
   0.0015708806458860636,
   -0.027661684900522232,
   -0.005147885996848345,

 Aqui se escribe el texto a consultar

In [35]:
results = db.query(
    query_texts=['dolor en extremidad superior'],
    n_results=2
)

In [36]:
results

{'ids': [['237', '7195']],
 'distances': [[0.33338966965675354, 0.7091445326805115]],
 'metadatas': [[{'_airbyte_ab_id': '339a9d69-2cc7-4d4f-85d6-1bf1d94765f4',
    '_airbyte_emitted_at': 1709771914974,
    'groupSideEffect': 'general',
    'ingles': 'Pain in extremity',
    'value': 'paininextremity'},
   {'_airbyte_ab_id': '851910ad-b58c-4bb3-a2b8-51a085b4c9b7',
    '_airbyte_emitted_at': 1709771914974,
    'groupSideEffect': 'general',
    'ingles': 'limb deformity',
    'value': 'limbdeformity'}]],
 'embeddings': None,
 'documents': [[None, None]],
 'uris': None,
 'data': None}

Aqui se consulta con 2 relaciones

In [37]:
results_where =db.query(
    query_texts=['dolor en extremidad superior'],
    n_results=2,
    where = { "$and":[ {
        'ingles' : {
            "$ne":'pain'
        }
    },
                       {
              'español' : {
          "$ne" : 'arm'
        }
    }] }
)

In [38]:
results_where

{'ids': [[]],
 'distances': [[]],
 'metadatas': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'data': None}